# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
town_df = pd.read_csv("../output_data/cities.csv", keep_default_na=False)
del town_df['Unnamed: 0']
town_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hilo,19.73,-155.09,73.40,73.0,90.0,4.70,US,06/30/20
1,port lincoln,-34.73,135.87,55.06,85.0,0.0,14.03,AU,06/30/20
2,bredasdorp,-34.53,20.04,62.60,72.0,14.0,9.17,ZA,06/30/20
3,barra do garcas,-15.89,-52.26,83.93,36.0,46.0,8.57,BR,06/30/20
4,ushuaia,-54.80,-68.30,28.40,79.0,40.0,4.70,AR,06/30/20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
# Store latitude and longitude in locations
locations = town_df[["Lat", "Lng"]]

# Set humidity as weight
humidity = town_df["Humidity"].astype(float)


# Plot Heatmap
figure = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=2)


# Add layer
figure.add_layer(heat_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
likely_cities = town_df.loc[(town_df["Humidity"]< 50)&
                             (town_df["Cloudiness"] < 20.0) &
                             (town_df["Max Temp"] > 60.0) & 
                              (town_df["Max Temp"] < 90.0),:]

print(f"There are {len(likely_cities)} cities with weather criteria. There are no null values.")

There are 39 cities with weather criteria. There are no null values.


In [26]:
likely_cities["Country"].value_counts().head()

BR    7
RU    5
NA    3
US    3
TM    2
Name: Country, dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = ideal_cities.copy()
hotel_df['Hotel Name']=''
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,choma,-16.81,26.95,69.91,31.0,0.0,9.01,ZM,06/30/20,
18,luangwa,-15.62,30.42,79.57,33.0,0.0,7.27,ZM,06/30/20,
27,saskylakh,71.92,114.08,82.04,36.0,0.0,7.40,RU,06/30/20,
44,zhigansk,66.77,123.37,70.09,47.0,0.0,10.20,RU,06/30/20,
46,diamantino,-14.41,-56.45,87.91,35.0,11.0,10.69,BR,06/30/20,


In [28]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (TypeError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [29]:
# Drop rows with empty strings
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(how = 'any',inplace = True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,choma,-16.81,26.95,69.91,31.0,0.0,9.01,ZM,06/30/20,Kamuboola Lodge
18,luangwa,-15.62,30.42,79.57,33.0,0.0,7.27,ZM,06/30/20,Gerasimos Safari Lodge
44,zhigansk,66.77,123.37,70.09,47.0,0.0,10.20,RU,06/30/20,Gostinichnyy Kompleks Aylgy
46,diamantino,-14.41,-56.45,87.91,35.0,11.0,10.69,BR,06/30/20,Hotel Delta
126,saldanha,-33.01,17.94,71.60,49.0,0.0,4.70,ZA,06/30/20,Blue Bay Lodge
132,kamyshin,50.10,45.42,72.12,47.0,19.0,5.59,RU,06/30/20,Guest House Aviator
135,luderitz,-26.65,15.16,78.80,24.0,0.0,4.70,NA,06/30/20,LÜDERITZ NEST HOTEL
176,lithakia,37.72,20.83,87.80,48.0,0.0,5.82,GR,06/30/20,Zante Plaza Hotel & Apartments
257,orahovac,42.40,20.65,82.40,28.0,18.0,10.29,XK,06/30/20,Esra Palace
264,alta floresta,-9.88,-56.09,87.80,45.0,0.0,3.36,BR,06/30/20,Floresta Amazonica Hotel


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(markers)


# Display figure
figure

Figure(layout=FigureLayout(height='420px'))